This example gets down all structures that have an ICSD ID in the Materials Project first.
It also uses multiprocessing to speed up the step of adding oxidation states. 

In [3]:
from pymatgen.analysis.structure_prediction.substitutor import Substitutor
from pymatgen import Specie, Element
from smact import ordered_elements
from itertools import product
from multiprocessing import Pool

from functions.structure_predictor import decorate_structures

In [4]:
import os
# You need to put your own MP API key in here
from pymatgen import MPRester
mp = MPRester(os.environ.get('MP_API_KEY'))

## Get a pool of oxidation state-decorated structures

In [5]:
# Get parent structures down from materials project
criteria = {'icsd_ids.0': {'$exists': True}}
properties = ['structure']
parent_structures = mp.query(criteria,properties)
structures_list = [i['structure'] for i in parent_structures]

100%|██████████| 48669/48669 [05:58<00:00, 81.73it/s]

In [111]:
decorated_strucs = decorate_structures(structures_list[:1000])
# Get rid of Nones for next step
decorated_strucs = [i for i in decorated_strucs if i is not None]

641  out of 1000 structures successfully decorated with oxidation states


## Set up some more preliminaries

In [112]:
# Set up substitutor
sub = Substitutor(threshold=1E-6)
# Get decorated structures into list of dicts (requried by substitutor)
decorated_strucs = [{'structure': i, 'id': n} for n,i in enumerate(decorated_strucs)]

In [113]:
def predict_struc_helper(species_set):
    suggested_strucs = sub.pred_from_structures(target_species=species_set,
                                               structures_list = decorated_strucs)
    return(suggested_strucs)

## Find crystal structures for a single set of species

In [114]:
# Use decorated structures as a pool for the structure predictor
species_of_interest = [Specie('Fe',3), Specie('Cu',2), Specie('S', -2)]
suggested_strucs = predict_struc_helper(species_of_interest)
print(" {} structures found!".format(len(suggested_strucs)))

 11 structures found!


## Find crystal structures for many species

In [115]:
# Generate a bunch of species combinations
metals, anions = ordered_elements(21,30), ['F', 'Cl', 'Br']
combos = list(product(metals,metals,anions))
combos = [i for i in combos if i[0] is not i[1]]

species_list = []
for i in combos:
    metal1, metal2 = Element(i[0]), Element(i[1])
    for j in product(metal1.common_oxidation_states, metal2.common_oxidation_states):
        sp1 = Specie(i[0], j[0])
        sp2 = Specie(i[1], j[1])
        sp3 = Specie(i[2], -1)
        species_list.append([sp1, sp2, sp3])
print("{} sets of species to consider".format(len(species_list)))

594 sets of species to consider


In [116]:
if __name__ == '__main__':
    with Pool(processes=8) as pool:
        all_predicted_structures = pool.map(predict_struc_helper, species_list)

# TODO save 